### LlamaParse

LlamaParse는 매달 무료 크레딧을 10000포인트 지급합니다. 이는 Default 모드를 기준으로 3000페이지에 해당합니다.

만약 보안이 중요하지 않은 경우에는, 가장 손쉽게 고품질의 추출 결과를 얻을 수 있는 강력한 API입니다.

https://cloud.llamaindex.ai/project/36c55af9-12df-4b1b-b6e4-0c99daa23754/parse

In [ ]:
# %pip install llama-parse

In [ ]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

In [3]:
import nest_asyncio
nest_asyncio.apply()

### 일반 Balance mode로 파싱

API에 어떤 파라미터도 설정하지 않으면, 자동으로 Balance mode로 파싱을 진행합니다.
Balance 모드는 대부분의 문서에 적합한 절충안으로, 1000 페이지에 $3입니다.

In [ ]:
# bring in deps
from llama_cloud_services import LlamaParse
import json
# set up parser
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    language="ko"
)

paper_path = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"
data_dir = "data"

md_json_objs = parser.get_json_result(paper_path)
json_dicts = md_json_objs[0]["pages"]
json.dumps(json_dicts, indent=4)

In [ ]:
for page in json_dicts:
    print(page["page"])
    print(page["md"])
    print("\n")

LlamaParse LlamaIndex라는 프레임워크를 만든 기업의 서비스이므로 Langchain의 Document 객체를 지원하지 않습니다. 

따라서 Langchain에 활용하고자 할 경우, 따로 page_content와 metadata를 넣어주어야합니다.

In [ ]:
# json_dicts를 langchain의 Document 객체로 변환
from langchain_core.documents import Document

# 페이지별로 Document 객체 생성
documents = []
for page in json_dicts:
    # 페이지 번호와 텍스트 내용을 추출
    page_num = page["page"]
    text_content = page["md"]
    
    # Document 객체 생성 (메타데이터에 페이지 번호 포함)
    doc = Document(
        page_content=text_content,
        metadata={"page": page_num, "source": paper_path}
    )
    
    # 생성된 Document 객체를 리스트에 추가
    documents.append(doc)

# 변환된 Document 객체 확인
print(f"총 {len(documents)}개의 Document 객체가 생성되었습니다.")
print(f"첫 번째 Document의 메타데이터: {documents[0].metadata}")


In [ ]:
for i in documents:
    print(i.page_content)
    print("\n")

### LlamaParse는 다양한 파싱 모드를 지원합니다. 

모든 파싱 모드는 기본적으로 ocr을 수행하는데, 이때 LLM을 활용해 재구성할지 말지를 설정할 수 있습니다.

parse_page_without_llm은 fast 모드로, 가장 빠른 파싱을 지원하며 with_llm, with_vlm, with_agent 순으로 점차 정확도를 높일 수 있으나 속도가 느려지고 비용이 올라갑니다.

복잡한 문서를 완전한 텍스트 파일로 변환하고자 하는 경우, parse_document_with_llm, parse_page_with_lvm, parse_page_with_agent를 사용하는 것을 권장합니다.

#### parse_page_with_llm으로 페이지별로 LLM 파싱

In [ ]:
# 필요한 라이브러리 가져오기
from llama_cloud_services import LlamaParse
import json

# LlamaParse 설정 - 다양한 파싱 모드 옵션
"""
LlamaParse 파싱 모드 옵션:
1. parse_page_without_llm: LLM 없이 빠른 파싱 (fast_mode=True와 동일)
2. parse_page_with_llm: 기본 모드, 페이지별 LLM 파싱 (balanced mode)
3. parse_document_with_llm: 전체 문서를 LLM에 전달하여 일관성 향상
4. parse_page_with_lvm: 멀티모달 비전 모델 사용 (use_vendor_multimodal_model=True)
5. parse_page_with_agent: 가장 정확한 에이전트 기반 파싱 (premium_mode=True)
"""

# 파서 설정 예시 (원하는 모드로 커스터마이징 가능)
parser = LlamaParse(
    result_type="markdown",  # "markdown" 또는 "text" 선택 가능
    language="ko",           # 언어 설정
    parse_mode="parse_page_with_llm",  # 파싱 모드 선택
    # 추가 옵션:
    # disable_image_extraction=False,  # 이미지 추출 비활성화 옵션
    # disable_ocr=False,               # OCR 비활성화 옵션
    # vendor_multimodal_model_name="openai-gpt4o"  # LVM 모델 선택 (parse_page_with_lvm 모드에서 사용)
)

# 파싱할 문서 경로 설정
paper_path = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"
data_dir = "data"

# 문서 파싱 및 결과 가져오기
md_json_objs = parser.get_json_result(paper_path)
json_dicts = md_json_objs[0]["pages"]
json.dumps(json_dicts, indent=4)

In [ ]:
for page in json_dicts:
    print(page["md"])

#### parse_page_with_lvm으로 멀티모달 파싱

In [ ]:
# 필요한 라이브러리 가져오기
from llama_cloud_services import LlamaParse
import json

# LlamaParse 설정 - 다양한 파싱 모드 옵션
"""
LlamaParse 파싱 모드 옵션:
1. parse_page_without_llm: LLM 없이 빠른 파싱 (fast_mode=True와 동일)
2. parse_page_with_llm: 기본 모드, 페이지별 LLM 파싱 (balanced mode)
3. parse_document_with_llm: 전체 문서를 LLM에 전달하여 일관성 향상
4. parse_page_with_lvm: 멀티모달 비전 모델 사용 (use_vendor_multimodal_model=True)
5. parse_page_with_agent: 가장 정확한 에이전트 기반 파싱 (premium_mode=True)
"""

# 파서 설정 예시 (원하는 모드로 커스터마이징 가능)
parser = LlamaParse(
    result_type="markdown",  # "markdown" 또는 "text" 선택 가능
    language="ko",           # 언어 설정
    parse_mode="parse_page_with_lvm",  # 파싱 모드 선택
    # 추가 옵션:
    # disable_image_extraction=False,  # 이미지 추출 비활성화 옵션
    # disable_ocr=False,               # OCR 비활성화 옵션
    vendor_multimodal_model_name="openai-gpt4o"  # LVM 모델 선택 (parse_page_with_lvm 모드에서 사용)
)

# 파싱할 문서 경로 설정
paper_path = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"
data_dir = "data"

# 문서 파싱 및 결과 가져오기
md_json_objs = parser.get_json_result(paper_path)
json_dicts = md_json_objs[0]["pages"]
json.dumps(json_dicts, indent=4)

In [ ]:
for page in json_dicts:
    print(page["md"])